In [3]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
from src.ffgen import ffgen
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [5]:
#initialize a table
rc = copy.deepcopy(utils.standardTable)
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop 
choices = []
for i in range (len(blockArray)):
    choices.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in choices:
            blockArray[i][key] = '*' 
#stitch amino acid blocks together to make a codon table
reductionistTable = {}
for i in range (len(blockArray)):
    reductionistTable.update(blockArray[i])
print(reductionistTable)

{'UUU': 'F', 'UUC': '*', 'UUA': '*', 'UUG': 'L', 'CUU': '*', 'CUC': '*', 'CUA': '*', 'CUG': '*', 'UCU': '*', 'UCC': 'S', 'UCA': '*', 'UCG': '*', 'AGU': '*', 'AGC': '*', 'UAU': '*', 'UAC': 'Y', 'UGU': 'C', 'UGC': '*', 'UGG': 'W', 'CCU': '*', 'CCC': '*', 'CCA': 'P', 'CCG': '*', 'CAU': 'H', 'CAC': '*', 'CAA': 'Q', 'CAG': '*', 'CGU': '*', 'CGC': '*', 'CGA': '*', 'CGG': 'R', 'AGA': '*', 'AGG': '*', 'AUU': 'I', 'AUC': '*', 'AUA': '*', 'AUG': 'M', 'ACU': '*', 'ACC': '*', 'ACA': 'T', 'ACG': '*', 'GUU': 'V', 'GUC': '*', 'GUA': '*', 'GUG': '*', 'AAU': 'N', 'AAC': '*', 'GCU': 'A', 'GCC': '*', 'GCA': '*', 'GCG': '*', 'GAU': 'D', 'GAC': '*', 'GAA': 'E', 'GAG': '*', 'GGU': 'G', 'GGC': '*', 'GGA': '*', 'GGG': '*', 'AAA': '*', 'AAG': 'K', 'UAA': '*', 'UAG': '*', 'UGA': '*'}
